In [1]:
# Find RL_Note path and append sys path
import os, sys
cwd = os.getcwd()
pos = cwd.find('RL_Note')
root_path = cwd[0:pos] + 'RL_Note'
sys.path.append(root_path)

In [2]:
import gym
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from pys.utils.memory import ReplayMemory

In [3]:
class DQN(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = Dense(24,activation='relu')
        self.fc2 = Dense(24,activation='relu')
        self.out = Dense(action_size,kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))

    def call(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.out(x)
        return q

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper-parameters for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        
        # Experience Replay
        self.batch_size = 32
        self.train_start = 500
        self.buffer_size = 5000
        self.memory = ReplayMemory(capacity=self.buffer_size)
        
        # Neural Network Architecture
        self.model        = DQN(self.state_size, self.action_size)
        self.target_model = DQN(self.state_size, self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.update_target_model()
        
    def remember(self, state, action, reward, next_state, done):
        state = np.reshape(state, [1, self.state_size])
        next_state = np.reshape(next_state, [1, self.state_size])
        transition = (state, action, reward, next_state, done)
        self.memory.append(transition)
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def choose_action(self,state):
        state = np.reshape(state, [1, self.state_size])
        # Exploration and Exploitation
        if (np.random.rand() <= self.epsilon):
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state))
        
    def train_model(self):
        # Train from Experience Replay
        # Training Condition - Memory Size
        if len(self.memory) < self.train_start:
            return 0.0
        # Decaying Exploration Ratio
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        # Sampling from the memory
        mini_batch = self.memory.sample(self.batch_size)
        
        states      = np.array([sample[0][0] for sample in mini_batch])
        actions     = np.array([sample[1] for sample in mini_batch])
        rewards     = np.array([sample[2] for sample in mini_batch])
        next_states = np.array([sample[3][0] for sample in mini_batch])
        dones       = np.array([sample[4] for sample in mini_batch])
        
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            q = self.model(states)
            one_hot_action = tf.one_hot(actions, self.action_size)
            q = tf.reduce_sum(one_hot_action * q, axis=1)
            
            target_q = tf.stop_gradient(self.target_model(next_states))
            
            max_q = np.amax(target_q, axis=-1)
            targets = rewards + (1 - dones) * self.discount_factor * max_q
            loss = tf.reduce_mean(tf.square(targets - q))
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))

        self.update_target_model()
        return loss

    def save_model(self):
        self.model.save_weights('./save_model/cartpole_dqn_TF', save_format='tf')

In [5]:
%matplotlib tk

ENV_NAME = 'CartPole-v1'
EPISODES = 1000
END_SCORE = 400

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = DQNAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores_avg, scores_raw, episodes, losses = [], [], [], []
    epsilons = []
    score_avg = 0
    
    end = False
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []

        state = env.reset()
        
        while not done:
            #env.render()

            # Interact with env.
            action = agent.choose_action(state)
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            loss = agent.train_model()
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | mem length: {:4d} | epsilon: {:.4f}'
                      .format(e, score_avg, len(agent.memory), agent.epsilon))

                # Save data for plot
                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                losses.append(np.mean(loss_list))
                epsilons.append(agent.epsilon)

                # View data
                plt.clf()
                plt.subplot(311)
                plt.plot(episodes, scores_avg, 'b')
                plt.plot(episodes, scores_raw, 'b', alpha=0.8, linewidth=0.5)
                plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                plt.title('LunarLanderv2 DQN')
                plt.subplot(312)
                plt.plot(episodes, epsilons, 'b')
                plt.xlabel('episode'); plt.ylabel('epsilon'); plt.grid()
                plt.subplot(313)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode'); plt.ylabel('losses') ;plt.grid()
                plt.savefig('./result/cartpole_dqn_TF.png')

                if score_avg > END_SCORE:
                    agent.save_model()
                    end = True
                    break
        if end == True:
            env.close()
            np.save('./save_model/data/cartpole_dqn_TF_epi',  episodes)
            np.save('./save_model/data/cartpole_dqn_TF_scores_avg',scores_avg)
            np.save('./save_model/data/cartpole_dqn_TF_scores_raw',scores_raw)
            np.save('./save_model/data/cartpole_dqn_TF_loss', losses)
            print("End")
            break

 length: 10000 | epsilon: 0.0100
epi: 693 | score avg 249.38 | mem length: 10000 | epsilon: 0.0100
epi: 694 | score avg 236.44 | mem length: 10000 | epsilon: 0.0100
epi: 695 | score avg 224.00 | mem length: 10000 | epsilon: 0.0100
epi: 696 | score avg 211.00 | mem length: 10000 | epsilon: 0.0100
epi: 697 | score avg 198.90 | mem length: 10000 | epsilon: 0.0100
epi: 698 | score avg 186.51 | mem length: 10000 | epsilon: 0.0100
epi: 699 | score avg 174.56 | mem length: 10000 | epsilon: 0.0100
epi: 700 | score avg 158.00 | mem length: 10000 | epsilon: 0.0100
epi: 701 | score avg 143.10 | mem length: 10000 | epsilon: 0.0100
epi: 702 | score avg 129.69 | mem length: 10000 | epsilon: 0.0100
epi: 703 | score avg 117.72 | mem length: 10000 | epsilon: 0.0100
epi: 704 | score avg 106.95 | mem length: 10000 | epsilon: 0.0100
epi: 705 | score avg 97.15 | mem length: 10000 | epsilon: 0.0100
epi: 706 | score avg 88.44 | mem length: 10000 | epsilon: 0.0100
epi: 707 | score avg 80.50 | mem length: 1000